In [ ]:
! pip install mediapipe

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp

In [ ]:
# 비디오가 있는 폴더 경로
# 상대경로 사용하면 좀 더 깔끔해요
root_dir = " " 

In [ ]:
# 파일 리스트 만들어주는 함수
def find_frame_dirs(root_dir):
    frame_dirs = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if any(f.lower().endswith((".jpg", ".png")) for f in filenames):
            frame_dirs.append(dirpath)
    return sorted(frame_dirs)

In [ ]:
# mediapipe사용하여 핸드 렌드마크 npz 생성
mp_hands = mp.solutions.hands

def extract_hands_for_folder(frames_dir: str, out_npz_path: str,
                             max_hands: int = 2):
    """
    frames_dir 안의 frame_*.jpg에 대해 MediaPipe Hands 수행.
    각 프레임마다 (max_hands, 21, 3) 랜드마크를 담아서 (N, max_hands*21*3) 배열로 저장.
    """
    frame_files = sorted([
        f for f in os.listdir(frames_dir)
        if f.lower().endswith((".jpg", ".png"))
    ])
    if not frame_files:
        print(f"[WARN] no images in {frames_dir}")
        return
    
    hands = mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=max_hands,
        min_detection_confidence=0.5
    )

    all_kps = []

    for fname in frame_files:
        img_path = os.path.join(frames_dir, fname)
        img_bgr = cv2.imread(img_path)
        if img_bgr is None:
            print(f"[WARN] failed to read {img_path}")
            continue
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        result = hands.process(img_rgb)

        # 기본 0으로 초기화: (max_hands, 21, 3)
        feat = np.zeros((max_hands, 21, 3), dtype=np.float32)

        if result.multi_hand_landmarks:
            for hi, hand_lms in enumerate(result.multi_hand_landmarks[:max_hands]):
                for li, lm in enumerate(hand_lms.landmark):
                    feat[hi, li, 0] = lm.x   # 0~1 정규화 좌표
                    feat[hi, li, 1] = lm.y
                    feat[hi, li, 2] = lm.z

        all_kps.append(feat.reshape(-1))  # (max_hands*21*3,)

    hands.close()

    all_kps = np.stack(all_kps, axis=0)  # (N, max_hands*21*3)
    os.makedirs(os.path.dirname(out_npz_path), exist_ok=True)
    np.savez_compressed(out_npz_path, hand_kps=all_kps)

    print(f"[extract_hands_for_folder] {frames_dir} -> {out_npz_path}, shape={all_kps.shape}")

In [ ]:
# 폴더 경로설정 확인용
'''text
프레임 폴더들:
   out_video\video_normal_001
   out_video\video_normal_002
   ...
'''
# 이런식으로 어떤 폴더가 있는지 나와야 해요
frame_dirs = find_frame_dirs(root_dir)
print("프레임 폴더들:")
for d in frame_dirs:
    print("  ", d)

In [ ]:
# "out_npz"는 npz를 담을 폴더 이름 또는 경로
hands_out_root = os.path.join(root_dir, "out_npz")
os.makedirs(hands_out_root, exist_ok=True)

for frames_dir in frame_dirs:
    sample_name = os.path.basename(frames_dir)  # 예: video_normal_001
    out_npz_path = os.path.join(hands_out_root, f"hands_{sample_name}.npz")
    extract_hands_for_folder(frames_dir, out_npz_path)